In [1]:
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup

In [2]:
#call this to retrieve company data
def retrieve_company_data(industry_code):
    base_url="https://www.reuters.com/sectors/industries/rankings?industryCode="
    url=base_url+str(industry_code)+"&page=-1"
    source_code=requests.get(url)
    text_code=source_code.content
    soup=BeautifulSoup(text_code,'html.parser')
    company_soup=soup.find(class_='column1 gridPanel grid8')
    All_company_section=company_soup.find(class_='dataSmall')
    All_company=All_company_section.findAll('tr')

    Tickers=[]
    Names=[]
    Market_Capitalization=[]
    Ttm=[]
    Employee=[]

    for company in All_company:
        company_td_all=company.findAll("td")
        for i,company_td in enumerate(company_td_all):
            if i==0:
                ticker=company_td.find('a')
                if ticker is not None:
                    Tickers.append(ticker.string)
            elif i==1:
                name=company_td.find('a')
                if name is not None:
                    Names.append(name.string)
            elif i==2:
                Market_Capitalization.append(company_td.string)
            elif i==3:
                Ttm.append(company_td.string)
            else:
                Employee.append(company_td.string)
            
    data=list(zip(Tickers,Names,Market_Capitalization,Ttm,Employee))
    return data
   
'''
    for n in Tickers:
        print(n)
    for n in Names:
        print(n)
'''
#t=retrieve_company_data(4)
#retrieve_people_data('9437.T')
 


'\n    for n in Tickers:\n        print(n)\n    for n in Names:\n        print(n)\n'

In [3]:
#call this to retrieve people data
def retrieve_people_data(ticker_no):
    base_url="https://www.reuters.com/finance/stocks/company-officers/"
    url=base_url+ticker_no
    source_code=requests.get(url)
    text_code=source_code.content
    soup=BeautifulSoup(text_code,'html.parser')
    people=soup.find(class_='column1 gridPanel grid8')
    All_people=people.find(class_='dataSmall')
    if All_people is None:
        return []
    All_people=All_people.findAll('tr')

    Name=[]
    Age=[]
    Since=[]
    Current_position=[]
    Descriptions=[]

    for person in All_people:
        person_td=person.findAll("td")
        for i,per_td in enumerate(person_td):
            name=per_td.find('a',class_='link')
            if name is not None:
                Name.append(name.string.strip())
            if i==1:
                Age.append(per_td.string)
            elif i==2:
                Since.append(per_td.string)
            elif per_td.string is not None:
                Current_position.append(per_td.string.strip())
    
    description=people.findAll(class_='dataSmall')[1]
    for about in description.find_all('tr'):
        about_td=about.findAll('td')
        for i,abt in enumerate(about_td):
            if i==1:
                Descriptions.append(abt.string.strip())
    return list(zip(Name,Age,Since,Current_position,Descriptions))
'''    for n in Name:
        print(n)
    for n in Age:
        print(n)
    for n in Since:
        print(n)
    for n in Descriptions:
        print(n)'''

'    for n in Name:\n        print(n)\n    for n in Age:\n        print(n)\n    for n in Since:\n        print(n)\n    for n in Descriptions:\n        print(n)'

In [4]:
#get the sector information
def getSectors(url):

    
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')

    sectors = []

    sectors_section=soup.find(id="tab1")
    sectors_section_table=sectors_section.find("tbody")
    sectors_tr=sectors_section_table.find_all("tr")
    for i in sectors_tr:
        sector_tag=i.find("td").find("a")
        sectors.append("https://www.reuters.com/"+sector_tag.get("href"))
    return sectors

#Parsing direct link
def scrap_first_link(url):
    # Get Source Code
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')

    data={}
    header = soup.find(class_="sectionRelatedTopics").find("ul")
    link=header.find("li")
    data[link.find("a").string]="https://www.reuters.com"+link.find("a").get("href")
    return data

#Parsing related links
def get_related_industries(url):

    # Get Source Code
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')

    related_industries_data = {}

    header=soup.find(class_="sectionRelatedTopics relatedIndustries").find("ul")
    for links in header.find_all("li"):
        related_industries_data[links.string]="https://www.reuters.com"+links.find("a").get("href")

    return related_industries_data

#this function will be called to get industry data
def getData():
    # Getting data of sectors
    sectors_data = getSectors("https://www.reuters.com/sectors/industries/significant?industryCode=4")
    links_data={}

    for sector in sectors_data:
        link=scrap_first_link(sector)
        links_data.update(link)


    links_data["Healthcare Reform"]="https://www.reuters.com/sectors/industries/overview?industryCode=151"

    related_links_data = {}
    #Parsing related links
    for key, value in links_data.items():
        link = get_related_industries(value)
        related_links_data.update(link)
        
    links_data.update(related_links_data)

    return links_data

In [5]:
#Given data in csv format
data=pd.read_csv('data.csv')

In [6]:
data

,ECONOMIC SECTOR,BUSINESS SECTOR,INDUSTRY GROUP,INDUSTRY,PermID,TRBC 2012 Hierarchical ID
0,Energy,NaN,NaN,NaN,4294952782,50
1,NaN,Energy - Fossil Fuels,NaN,NaN,4294952999,5010
2,NaN,NaN,Coal,NaN,4294952998,501010
3,NaN,NaN,NaN,Coal,4294952997,50101010
4,NaN,NaN,Oil & Gas,NaN,4294952996,501020
5,NaN,NaN,NaN,Integrated Oil & Gas,4294952995,50102010
6,NaN,NaN,NaN,Oil & Gas Exploration and Production,4294952994,50102020
7,NaN,NaN,NaN,Oil & Gas Refining and Marketing,4294952993,50102030
8,NaN,NaN,Oil & Gas Related Equipment and Services,NaN,4294952992,501030
9,NaN,NaN,NaN,Oil & Gas Drilling,4294952991,50103010


In [7]:
Industry=[]
PermID=[]
New_Data=[]
for index,d in data.iterrows():
    if d['INDUSTRY'] is not None:
        Industry.append(d['INDUSTRY'])
        PermID.append(d['PermID'])
New_Data=list(zip(Industry,PermID))
Data_Dictionary=dict(New_Data)
Data_Dictionary

{nan: 4294952813,
 'Coal': 4294952997,
 'Integrated Oil & Gas': 4294952995,
 'Oil & Gas Exploration and Production': 4294952994,
 'Oil & Gas Refining and Marketing': 4294952993,
 'Oil & Gas Drilling': 4294952991,
 'Oil Related Services and Equipment': 4294952990,
 'Oil & Gas Transportation Services': 4294952811,
 'Renewable Energy Equipment & Services': 4294952809,
 'Renewable Fuels': 4294952808,
 'Uranium': 4294952775,
 'Chemicals - Commodity': 4294952986,
 'Agricultural Chemicals': 4294952985,
 'Specialty Chemicals': 4294952984,
 'Diversified Chemicals': 4294952983,
 'Precious Metals & Minerals': 4294952771,
 'Steel': 4294952979,
 'Aluminum': 4294952978,
 'Specialty Mining & Metals': 4294952977,
 'Gold': 4294952770,
 'Mining Support Services & Equipment': 4294952769,
 'Diversified Mining': 4294952768,
 'Construction Materials': 4294952975,
 'Forest & Wood Products': 4294952972,
 'Paper Products': 4294952971,
 'Non-Paper Containers & Packaging': 4294952969,
 'Paper Packaging': 4294952

In [10]:
#parsing the all sector data
sector=getData()

In [11]:
Industry_Data=[]
for industry,permid in Data_Dictionary.items():
    if industry in sector:
        Industry_Data.append(tuple((industry,permid,sector[industry])))     
    else :
        Industry_Data.append(tuple((industry,permid,'None')))


In [12]:
df=pd.DataFrame(Industry_Data)

In [15]:
df=df.drop(0)


ValueError: labels [0] not contained in axis

In [17]:
df.to_csv('Industry_Data.csv',header=['Industry','PermID','Link'])
df


In [18]:
read_industry_data=pd.read_csv('Industry_Data.csv')
read_industry_data

,Unnamed: 0,Industry,PermID,Link
0,1,Coal,4294952997,https://www.reuters.com/sectors/industries/ove...
1,2,Integrated Oil & Gas,4294952995,https://www.reuters.com/sectors/industries/ove...
2,3,Oil & Gas Exploration and Production,4294952994,https://www.reuters.com/sectors/industries/ove...
3,4,Oil & Gas Refining and Marketing,4294952993,https://www.reuters.com/sectors/industries/ove...
4,5,Oil & Gas Drilling,4294952991,https://www.reuters.com/sectors/industries/ove...
5,6,Oil Related Services and Equipment,4294952990,https://www.reuters.com/sectors/industries/ove...
6,7,Oil & Gas Transportation Services,4294952811,https://www.reuters.com/sectors/industries/ove...
7,8,Renewable Energy Equipment & Services,4294952809,https://www.reuters.com/sectors/industries/ove...
8,9,Renewable Fuels,4294952808,https://www.reuters.com/sectors/industries/ove...
9,10,Uranium,4294952775,https://www.reuters.com/sectors/industries/ove...


In [19]:
#Collecting Industries Detail

In [24]:
Industry_Info_Data=[]
for i,d in enumerate(Industry_Data):
    if d[2]!='None':
        #print(d[2])
        code=d[2].split("=")[1].strip()
        #print(code)
        c_data=retrieve_company_data(code)
       # print(c_data)
        if c_data is not None:
            for c in c_data:
                single_industry_val=tuple((d[0],d[1],c[0],c[1],c[2],c[3],c[4]))
                Industry_Info_Data.append(single_industry_val)
Industry_Info_Data

SSLError: ("bad handshake: SysCallError(10054, 'WSAECONNRESET')",)

In [25]:
df=pd.DataFrame(Industry_Info_Data)

In [26]:
df.to_csv('Industry_Info_Data.csv',header=['Industry','PermID','Ticker','Name','Market Capitalisation','ttm','Employees'])

In [27]:
df=pd.read_csv('Industry_Info_Data.csv')
df

,Unnamed: 0,Industry,PermID,Ticker,Name,Market Capitalisation,ttm,Employees
0,0,Coal,4294952997,GLEN.L,Glencore PLC,"72,503,169.715","34,901,548.957","19,992"
1,1,Coal,4294952997,3315.T,Nippon Coke & Engineering Co Ltd,"52,982.680","140,024.594","154,832"
2,2,Coal,4294952997,1518.T,Mitsui Matsushima Co Ltd,"37,188.980","97,783.000","1,020"
3,3,Coal,4294952997,1514.T,Sumiseki Holdings Inc,"18,486.131","57,474.000","1,288"
4,4,Coal,4294952997,8835.T,Taiheiyo Kouhatsu Inc,"9,952.893","12,767.000",57
5,5,Coal,4294952997,CNX.N,CONSOL Energy Inc,"7,907.983","33,164.000",860
6,6,Coal,4294952997,BTU.N,Peabody Energy Corp,"3,754.730","2,374.191","2,307"
7,7,Coal,4294952997,ARCH.N,Arch Coal Inc,"3,100.687","5,502.500","6,700"
8,8,Coal,4294952997,CNTE.PK,Contura Energy Inc,"1,757.655","2,276.834","4,025"
9,9,Coal,4294952997,FELP.N,Foresight Energy LP,607.218,--,"2,300"


In [28]:
Person_Data=[]
for i,d in enumerate(Industry_Info_Data):
    if d[2] is not None:
        #print(d[2])
        p_data=retrieve_people_data(d[2])
        if p_data is not None:
            for p in p_data:
                Person_Data.append(tuple((d[0],d[1],d[2],p[0],p[1],p[2],p[3],p[4])))
Person_Data

SSLError: ("bad handshake: SysCallError(10054, 'WSAECONNRESET')",)

In [29]:
df=pd.DataFrame(Person_Data)

In [33]:
df.to_csv('Person_Data.csv',header=['Industry','PermID','Ticker','Name','Age','Since','Current_Position','Descriptions'],encoding='utf-8')

In [34]:
df=pd.read_csv('Person_Data.csv',encoding='utf-8')
df

,Unnamed: 0,Industry,PermID,Ticker,Name,Age,Since,Current_Position,Descriptions
0,0,Coal,4294952997,GLEN.L,Anthony Hayward,59.0,2013.0,Non-Executive Independent Chairman of the Board,Dr. Anthony B. Hayward is Non-Executive Indepe...
1,1,Coal,4294952997,GLEN.L,Ivan Glasenberg,60.0,2011.0,"Chief Executive Officer, Executive Director",Mr. Ivan Glasenberg is Chief Executive Officer...
2,2,Coal,4294952997,GLEN.L,Steven Kalmin,47.0,2005.0,Chief Financial Officer,Mr. Steven Kalmin is a Chief Financial Officer...
3,3,Coal,4294952997,GLEN.L,John Burton,52.0,2011.0,Company Secretary,Mr. John Burton is Company Secretary of Glenco...
4,4,Coal,4294952997,GLEN.L,Peter Coates,71.0,2014.0,Non-Executive Director,Mr. Peter Roland Coates is a Non-Executive Dir...
5,5,Coal,4294952997,GLEN.L,Leonhard Fischer,54.0,2011.0,Independent Non-Executive Director,Mr. Leonhard H. Fischer is an Independent Non-...
6,6,Coal,4294952997,GLEN.L,Martin Gilbert,62.0,2017.0,Non-Executive Independent Director,"Mr. Martin James Gilbert, LL.D., has been appo..."
7,7,Coal,4294952997,GLEN.L,Peter Grauer,71.0,2014.0,Senior Non-Executive Independent Director,Mr. Peter T. Grauer is an Senior Non-Executive...
8,8,Coal,4294952997,GLEN.L,John Mack,72.0,2013.0,Independent Non-Executive Director,Mr. John J. Mack is an Independent Non-Executi...
9,9,Coal,4294952997,GLEN.L,Patrice Merrin,68.0,2014.0,Independent Non-Executive Director,Ms. Patrice E. Merrin is an Independent Non-Ex...
